# IMPORT

In [1]:
import os, shutil #Controlling Files and paths and folder control
from shutil import copyfile
from pdf2image import convert_from_path #Libary convert pdf file to img file
from google.oauth2 import service_account #Control API Keys
from google.cloud import vision # Vision API from Google
import io
import string, random
import pandas as pd
# Imports the Google Cloud client library
from google.cloud import language_v1
from google.cloud.language_v1 import enums

In [2]:
#from __future__ import unicode_literals, print_function
#from spacy.lang.en import English # updated

# PDF TO TEXT

In [3]:
#Paths that have all folders which use everything
pdfIMGPopplerPath = '/Users/kunal/Documents/VdartResumeProject/Poppler/poppler-0.68.0_x86/poppler-0.68.0/bin/'
imgTxtVisionAPIPath = "/Users/kunal/Documents/VdartResumeProject/APIKEYSGOOGLE/resumeMatcher-pdf2img.json"
runningDocumentPath = '/Users/kunal/Documents/VdartResumeProject/runningDoc/'
sourceFolderResumesPath = '/Users/kunal/Documents/VdartResumeProject/50_resumes/'
excelFilesPath = '/Users/kunal/Documents/VdartResumeProject/ExcelFiles/'
nlpAutoAPIPath = "/Users/kunal/Documents/VdartResumeProject/APIKEYSGOOGLE/resumeMatcher-NLP_create_data.json"
jsonFolderPath = '/Users/kunal/Documents/VdartResumeProject/JSONLFILES/'

In [4]:
def convert_pdf_2_image(uploaded_image_path, uploaded_image):
    #Using the convert_from_path function
    #Same name as pdf but converted to img
    #Watch out for poppler -- necceasary to function
    os.chdir(uploaded_image_path) # Change the working diretory to path that contains the PDF file
    file_name = str(uploaded_image).replace('.pdf','') # file name for png still going to get changed later
    pages = convert_from_path(uploaded_image, 200,poppler_path=pdfIMGPopplerPath) #function to change pdf to img
    pageNumCount = 1 #numbering for all the different images if pdf is multiple pages
    outputNames = []
    for page in pages:
        output_file = file_name+"_"+str(pageNumCount) + '.jpg'#uptaded name for image
        page.save(output_file, 'JPEG')#save img
        pageNumCount +=1
        outputNames.append(output_file)
    return outputNames #names of img

In [5]:
def convert_img_2_text(imagePath):
    #Using API from Google
    #Returns a JSON file but text is extracted from it
    keyDIR = imgTxtVisionAPIPath #JSON key file to call the api
    credentials = service_account.Credentials.from_service_account_file(keyDIR) #using service account to go through google
    client = vision.ImageAnnotatorClient(credentials=credentials) # client api
    with io.open(imagePath, 'rb') as image_file: #opening and reading img
        content = image_file.read() 
    image = vision.types.Image(content=content) # calling and running the client Visison API
    response = client.text_detection(image=image) # JSON return with all values
    texts = response.text_annotations # Go through Json and extract the text detected
    totalString = ''
    for text in texts:
        totalString+=text.description
    totalString = totalString.rsplit(' ', 1)[0] # throws away all the metadata with location -- could be used later
    return totalString #TEXT detected

In [6]:
def deleteIMG(path):
    #Deletes all the images from a folder
    for i in os.listdir(path):
        if i.endswith(".jpg"):
            os.remove(folder_pdf + i)
def deleteEverythingInFolder(folder_pdf):
    #deletes everything in the folder including folders and files
    for file in os.listdir(folder_pdf):
        try:
            shutil.rmtree(folder_pdf+ file) #remove folder
        except NotADirectoryError:
            try:
                os.remove(folder_pdf+ file) # if it is not a folder than it is a file so it removes it also
            except:
                pass
def moveRenameAllFiles(sourceFolderResumes, folder_pdf):
    #Moves all files from source and renames them starting from 100
    #If start from 1 then the ordering will be off
    for realFile in os.listdir(sourceFolderResumes):
        copyfile(sourceFolderResumes + realFile, folder_pdf + realFile) #duplicates all the files from source
    os.chdir(folder_pdf) # move them to the running folder
    documentcounter = 100 # names "Dociment_[num].pdf" start from 100 cause that is how numbering is listed correctly
    for i in os.listdir(folder_pdf):
        fileNameCreator = "Document_" + str(documentcounter) + ".pdf"
        print(i + "   changed to  " + fileNameCreator)
        os.rename(i, fileNameCreator)  # change name of file
        documentcounter+=1

In [7]:
folder_pdf = runningDocumentPath # set var
sourceFolderResumes = sourceFolderResumesPath

In [8]:
deleteEverythingInFolder(folder_pdf)
moveRenameAllFiles(sourceFolderResumes, folder_pdf)
numberOfFiles = len([name for name in os.listdir(folder_pdf) if os.path.isfile(name)]) 
#PNGFilesExist = checkifPNGExists(folder_pdf)
#deletes everything in the folder, copy and moves all files and renames them starting from 100

Document_402.pdf   changed to  Document_100.pdf
Document_403.pdf   changed to  Document_101.pdf
Document_405.pdf   changed to  Document_102.pdf
Document_406.pdf   changed to  Document_103.pdf
Document_407.pdf   changed to  Document_104.pdf
Document_408.pdf   changed to  Document_105.pdf
Document_409.pdf   changed to  Document_106.pdf
Document_410.pdf   changed to  Document_107.pdf
Document_411.pdf   changed to  Document_108.pdf
Document_412.pdf   changed to  Document_109.pdf
Document_413.pdf   changed to  Document_110.pdf
Document_414.pdf   changed to  Document_111.pdf
Document_415.pdf   changed to  Document_112.pdf
Document_417.pdf   changed to  Document_113.pdf
Document_418.pdf   changed to  Document_114.pdf
Document_419.pdf   changed to  Document_115.pdf
Document_420.pdf   changed to  Document_116.pdf
Document_421.pdf   changed to  Document_117.pdf
Document_422.pdf   changed to  Document_118.pdf
Document_423.pdf   changed to  Document_119.pdf
Document_424.pdf   changed to  Document_

In [9]:
#Asks the user which document they want to run and if they don't input anything, its a random document
doubleCheckDocumentInputCount = 0
while True:
    documentRun = input("Which Document do you wanna run?")
    if documentRun == "" and doubleCheckDocumentInputCount >= 1:
        documentRun = (random.choice(os.listdir(folder_pdf)))
        print("You choose RANDOM so your document is " + documentRun)
        break
    elif documentRun == "":
        print("This is a double check if you want a random document?")
        doubleCheckDocumentInputCount+=1
    elif documentRun == "BREAK":
        print("Exiting")
        documentRun = "INVALID"
        break
    else:
        documentRun = "Document_" + documentRun + ".pdf"
        print(os.path.isfile(folder_pdf + documentRun))
        if os.path.isfile(folder_pdf + documentRun):
            print("You choose a specific number so your document is " + documentRun)
            break
        else:
            print("Document was invalid. Input >>> BREAK <<< to stop")
            continue

Which Document do you wanna run?125
True
You choose a specific number so your document is Document_125.pdf


In [10]:
#Makes a folder and create images of each page in PDF document
textList = []
numberPagesList = []
for i in os.listdir(folder_pdf): 
    if i == documentRun:# only runs the file that was choosen by user
        os.chdir(folder_pdf) 
        folderNameCreate = folder_pdf+"Fold_"+i[:-4]+"/" 
        print(folderNameCreate)
        os.mkdir(folderNameCreate) #Create folder
        shutil.move(i,folderNameCreate) #MovePDF into folder
        os.chdir(folderNameCreate)
        for j in os.listdir(folderNameCreate):
            print(j)
            if j.endswith(".pdf"): 
                imgName = convert_pdf_2_image(folderNameCreate, j) #call pdf2img function
                print("Running Number: " + str(len(imgName))) 
                printString = ""
                for docName in imgName: # for making the printing better
                    printString += docName + " " 
                print(printString) 
            for q in range(len(imgName)):
                text = convert_img_2_text(folderNameCreate + i[:-4] + "_" + str(q+1) + ".jpg")
                textList.append(text)     #Convert to text and append all the text to list
                numberPagesList.append(len(imgName))   
    else:
        os.chdir(folder_pdf) #delete any other file that exists
        os.remove(i)

/Users/kunal/Documents/VdartResumeProject/runningDoc/Fold_Document_125/
Document_125.pdf
Running Number: 2
Document_125_1.jpg Document_125_2.jpg 


## Convert to Excel

In [11]:
df = pd.DataFrame()  # creates a tabel with all the text and pages
df['Text Extracted'] = textList
df['DocumentNum'] = numberPagesList

In [12]:
os.chdir(excelFilesPath)

In [13]:
EXCELFILENAME = documentRun[:-4] + "_text.xlsx"
df.to_excel(EXCELFILENAME, index = False) # save the text to the excel file folder

## Read Excel

In [14]:
EXCELFILENAME = documentRun[:-4] + "_text.xlsx" # read the same file

In [15]:
os.chdir(excelFilesPath) # change working path to excel files folder

In [16]:
text = pd.ExcelFile(EXCELFILENAME) 
dftext = text.parse("Sheet1") # read the file to see what was saved

In [17]:
dftext

,Text Extracted,DocumentNum
0,"Benjamin Meena\nDenver, CO 80202\nWeb Develope...",2
1,Managed a new and used vehicle inventory of gr...,2


In [18]:
text_list = dftext["Text Extracted"].tolist() # converts both columns to lists 
pagesList = dftext["DocumentNum"].tolist()

## Check for Recommendation letters

In [19]:
count=0
numb = 0
recNumList=[]
for i in text_list: # for all the text files scan if the word recommendation is in there
    if "Recommendation" in str(i): 
        recNumList.append(numb)
    numb+=1

In [20]:
recNumList # print all the page numbers with the word in it

[]

# Converts the text files to an array but still split into pages

In [21]:
runningTextFINAL = '\n'.join(textList) # the text is combined

# Create Sentences

In [22]:
print(runningTextFINAL) 

Benjamin Meena
Denver, CO 80202
Web Developer
Github: bmeena57.github.io
LinkedIn: linkedin.com/in/benjaminmeena
(757) 234-5481
ben.meena16@gmail.co
m
EXPERIENCE
Technical Highlights
HTML
- Certificate of Web
The Tech Academy, Denver
Development
CSS
JavaScript
AJAX
jquery
November 2019 - March 2020
Software Development
REACT
Bootstrap
Comprehensive overview of software development with courses in HTML5,
CSS3, JavaScript/jQuery, SQL/MS SQL Server, MVC, Source Control (Git),
and Agile product development methods (Scrum).
Non-Technical Highlights
Participated in a 2 week long live project with google meetings; updating
web pages for their client.
Self-driven
Strong Leadership
5 Guys Enterprise, Greenwood Village-Manager
November 2019- PRESENT
Management
Time efficient
Ensure that the Occupational Safety and Health Act, local health and safety codes,
and company safety and security policies are followed.
Coachable
Assist with recruiting, interviewing, and hiring of Crew Members
Sociable
Wo

In [23]:
textChoosen = runningTextFINAL # set variables

In [24]:
setnecnceAuto = []
for line in textChoosen.splitlines(): # function that splits all the text by lines, this is all automatic and not accurate
    setnecnceAuto.append(line)

In [25]:
print("CHAR: " + str(len(textChoosen)) + ", LINES: " + str(len(setnecnceAuto))) # print some information about the text

CHAR: 1636, LINES: 59


In [26]:
sentenceList= setnecnceAuto #var names

# Automatic Entity Creation

In [27]:
#https://cloud.google.com/natural-language/docs/basics#entity_analysis
def extractMajorEntities(text_content, salienceScoreThres):
    keyDIR = nlpAutoAPIPath # JSON path with the crediatials 
    credentials = service_account.Credentials.from_service_account_file(keyDIR) # same as before api credentials
    client = language_v1.LanguageServiceClient(credentials=credentials)  # language client 
    type_ = enums.Document.Type.PLAIN_TEXT 
    language = "en" # english text
    document = {"content": text_content, "type": type_, "language": language} #formatting the design of api input -- leave it
    encoding_type = enums.EncodingType.UTF8 # reading the document -- must have 
    response = client.analyze_entities(document, encoding_type=encoding_type) # actual run of the api
    majorValues = []
    for entity in response.entities:  #reading the JSON that was returned by analyze entities
        if entity.salience > salienceScoreThres: #salience score is the importance of the entity compared to the full document
            majorValues.append([entity.name,entity.salience]) # if it is greater than the threshold then save it and return it
    return  majorValues
def extractEntities(text_content):
    keyDIR = nlpAutoAPIPath# JSON path with the crediatials 
    credentials = service_account.Credentials.from_service_account_file(keyDIR) # same as before api credentials
    client = language_v1.LanguageServiceClient(credentials=credentials) # language client
    type_ = enums.Document.Type.PLAIN_TEXT
    language = "en"# english text
    document = {"content": text_content, "type": type_, "language": language}#formatting the design of api input -- leave it
    encoding_type = enums.EncodingType.UTF8# reading the document -- must have 
    response = client.analyze_entities(document, encoding_type=encoding_type) # actual run of the api
    nonuseEntities = ["OTHER", "NUMBER"] # attributes of the entities that dont matter and are of no use
    allEntitiesExtracted = []
    for entity in response.entities: # loop through json and see all entities
        currentRunningEntity = []
        #print(entity.name)
        #print(enums.Entity.Type(entity.type).name)
        if enums.Entity.Type(entity.type).name not in nonuseEntities: # check if meta data from entity is important
            #print("Detected " + entity.name + " as " + enums.Entity.Type(entity.type).name)
            currentRunningEntity.append(entity.name)
            currentRunningEntity.append(enums.Entity.Type(entity.type).name)
            allEntitiesExtracted.append(currentRunningEntity) # append it to list 
        else: 
            for mention in entity.mentions:
                if enums.EntityMention.Type(mention.type).name == "PROPER":
                    #print("Detected " + mention.text.content + " as " + enums.EntityMention.Type(mention.type).name)
                    currentRunningEntity.append(mention.text.content)
                    currentRunningEntity.append(enums.EntityMention.Type(mention.type).name)
                    allEntitiesExtracted.append(currentRunningEntity) # another check if it entity is important 
                    # must see format or return entity to see how this is working 
        #allEntitiesExtracted.append(currentRunningEntity) if currentRunningEntity != [] else print("-")
        #allEntitiesExtracted.append(currentRunningEntity)   
    finalArray = []
    runningEnitityCount = 0
    positionValuesEnitiesList = []
    for i in range(len(allEntitiesExtracted)):
        arrayrun = []
        arrayrun.append(allEntitiesExtracted[i][0])
        arrayrun.append(allEntitiesExtracted[i][1])
        arrayrun.append(text_content.find(allEntitiesExtracted[i][0]))
        positionValuesEnitiesList.append(arrayrun)
    positionValuesEnitiesList = sorted(positionValuesEnitiesList, key=lambda x: x[2]) # sort everything by what is first 
    #print(positionValuesEnitiesList)
    # some of the entities should be combined together becuase they are 2 words and they both fit the position of what the 
    # entity is so, the following code checks the distance between the 2 words and see if they should be together and if so 
    # it return with the entire phrase as a entity 
    runningPositionValuesCount = 0 
    for numb in range(len(positionValuesEnitiesList)-1):
        distancebetween = 0
        lenWord = len(positionValuesEnitiesList[numb][0])
        wordPosition = positionValuesEnitiesList[numb][2]
        distancebetween =((positionValuesEnitiesList[numb+1][2] - (wordPosition + lenWord)))
        sameElement = True if positionValuesEnitiesList[numb][1] == positionValuesEnitiesList[numb+1][1] else False
        #print(sameElement)
        if distancebetween < 2.5 and sameElement: # 2.5 is distance
            #print(positionValuesEnitiesList[numb])
            returnArray = [text_content[positionValuesEnitiesList[numb][2]:positionValuesEnitiesList[numb+1][2]+len(positionValuesEnitiesList[numb+1][1])]]
            returnArray.append(positionValuesEnitiesList[numb][1])
            finalArray.append(returnArray)
        else: 
            finalArray.append([positionValuesEnitiesList[numb][0], positionValuesEnitiesList[numb][1]])
    #print(distancebetween)
    #if distancebetween/len(positionValuesEnitiesList)-1 < 2.5:
        #returnArray = [text_content[positionValuesEnitiesList[0][2]:positionValuesEnitiesList[-1][2]+len(positionValuesEnitiesList[-1][1])]]
        #returnArray.append(positionValuesEnitiesList[0][1])
        #print(returnArray)
    return finalArray

In [28]:
#run everything above 
sentenceArrayWithEntities = []
for i in sentenceList:
    temparray = []
    temparray.append(i)
    entityArray = extractEntities(i)
    for j in entityArray:
        temparray.extend(j)
    sentenceArrayWithEntities.append(temparray)

In [29]:
importantEntitiesArray = extractMajorEntities(runningTextFINAL, 0.05)
print(importantEntitiesArray)

[['LinkedIn: linkedin.com/in/benjaminmeena', 0.15112082660198212], ['Web', 0.07992628961801529]]


# Check what document is about

In [30]:
def sample_classify_text(text_content):
    keyDIR = nlpAutoAPIPath # same as before api credentials
    credentials = service_account.Credentials.from_service_account_file(keyDIR) # same as before
    client = language_v1.LanguageServiceClient(credentials=credentials)
    type_ = enums.Document.Type.PLAIN_TEXT
    language = "en"
    document = {"content": text_content, "type": type_, "language": language}
    response = client.classify_text(document)
    # Loop through classified categories returned from the API
    categoryList = []
    # this api returns what category the entire document is about and see what it really means
    for category in response.categories:
        # Get the name of the category representing the document.
        # See the predefined taxonomy of categories:
        # https://cloud.google.com/natural-language/docs/categories
        # print(u"Category name: {}".format(category.name))
        categoryList.append([category.name, category.confidence])
    return categoryList

In [31]:
allPossibleCategory = sample_classify_text(runningTextFINAL)
for category in allPossibleCategory:
    print(("The Document is about: " + '\033[1m' + category[0] + '\033[0m' + 
           " \t with a " + '\033[1m' + "{:.1f}"+ '\033[0m' + " accuracy.\n").format(category[1]*100))
#print(allPossibleCategory)
# print in a bold way

The Document is about: /Internet & Telecom/Web Services/Web Design & Development 	 with a 62.0 accuracy.

The Document is about: /Jobs & Education/Jobs/Job Listings 	 with a 60.0 accuracy.

The Document is about: /Science/Computer Science 	 with a 56.0 accuracy.

The Document is about: /Computers & Electronics/Programming 	 with a 52.0 accuracy.



# Save Sentences and Entities|

In [32]:
os.chdir(excelFilesPath)
# saved what was created in the excel folder place but with the following name

In [33]:
df = pd.DataFrame(sentenceArrayWithEntities)
EXCELFILENAMESent = documentRun[:-4] + "_sentencesNew.xlsx"
df.to_excel(EXCELFILENAMESent)

In [34]:
EXCELFILENAMESent
# name of the file with the sentences

'Document_125_sentencesNew.xlsx'

# --- HUMAN MUST MANUALLY INPUT ENTITIES ---
**CHECK THIS LINK FOR MORE INSTRUCTIONS**

https://docs.google.com/document/d/1BdbWL7ePGPYiSVa07g8Erknwm_Zmi2JUwp588vJdUug/edit?usp=sharing

# Convert Array to JSON

In [61]:
EXCELFILENAMESent = documentRun[:-4] + "_sentencesNew.xlsx"
# reads the excel name

## Extract Excel and Turn into a list

In [62]:
os.chdir(excelFilesPath)

In [63]:
text = pd.ExcelFile(EXCELFILENAMESent)
totalTextRunning = runningTextFINAL
attributes = text.parse("Sheet1") 
# read excel

In [64]:
attributes
# this is a table of all of the entities. 
# IMPORTANT -- the name of the first row with the text must be named "Text"
# could be every how long columns

,text,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Benjamin Meena,Name,All,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Denver, CO 80202",ADDRESS,All,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Web Developer,WorkExperience,All,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Github: bmeena57.github.io,Link,bmeena57.github.io,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LinkedIn: linkedin.com/in/benjaminmeena,Link,linkedin.com/in/benjaminmeena,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,(757) 234-5481,Phone,All,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ben.meena16@gmail.com,Link,All,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,EXPERIENCE,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Technical Highlights,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,HTML,Software,All,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
RowList = attributes.values.tolist()
attributeList = [[i for i in row if isinstance(i,str)] for row in RowList]
#Converts to a 2d array 

## Create Final List

In [66]:
def sentence_2_word(sentence):
    # this functuion takes a sentence and splits it into each word and returns a list of all the words
    word = ""
    listofWords = []
    for i in sentence:
        if not (i == " " or i == "\n"):
            word += i 
        else:
            listofWords.append(word)
            word = ""
    if len(listofWords) == 0:
        listofWords.append(sentence)
    return listofWords

In [67]:
def find_all_indexes(input_str, search_str):
    # find all the positions of where a text is in a string
    # not used so dont worry
    l1 = []
    length = len(input_str)
    index = 0
    while index < length:
        i = input_str.find(search_str, index)
        if i == -1:
            return l1
        l1.append(i)
        index = i + 1
    return l1[0]

In [68]:
def checkNotWorkingValues(textFindPosition, totalTextRunning):
    # if the find function cant find the text in the total text then this function splits the search text into each word 
    # find its value out of the entire text and if they are close enough then then it returns the position of the starting 
    # word of the search text but if it can't get a definitie answer then it returns -1 
    words = textFindPosition.split()
    wordPositions = []
    for singleWord in words:
        wordPositions.append(totalTextRunning.find(singleWord))
    positionsCount = 0
    numCorrect = 0
    for num in range(len(wordPositions)-1):
        if wordPositions[num]+len(words[num])+1 == wordPositions[num+1]:
            numCorrect+=1
        positionsCount+=1
    if numCorrect == len(words)-1:
        return wordPositions[0]
    else:
        return -1

In [69]:
def checkNotWorkingValues(textFindPosition, totalTextRunning):
    words = textFindPosition.split()
    if len(words) >= 7:
        wordListNew = []
        for number in range(1, len(words), 2):
            wordListNew.append(words[number-1] + " " + words[number])
        if not len(words) % 2 == 0: 
            wordListNew.append(words[-1])
        #print(wordListNew)
        words = wordListNew
    wordPositions = []
    numberstarting = 0  
    for singleWord in words:
        #print(totalTextRunning[numberstarting:].find(singleWord))
        wordPositions.append(totalTextRunning[numberstarting:].find(singleWord)+numberstarting)
        if not totalTextRunning[numberstarting:].find(singleWord) == -1:
            numberstarting = totalTextRunning[numberstarting:].find(singleWord) +numberstarting
        #print(numberstarting)    
    #print(wordPositions)
    positionsCount = 0
    numCorrect = 0
    for num in range(len(wordPositions)-1):
        if abs(wordPositions[num]+len(words[num])+1 - wordPositions[num+1]) < 2:
            numCorrect+=1
        #else:
            #print(words[num])
            #print(wordPositions[num])
            #print(len(words[num])+1)
            #print(wordPositions[num]+len(words[num])+1)
            #print(wordPositions[num+1])
            #print("")
        positionsCount+=1
    #print(positionsCount)
    #print(numCorrect)
    if not positionsCount == 0:
        return wordPositions[0] if numCorrect/positionsCount > 0.6 else -1
    else:
        return -1

In [70]:
numbAttributeReal = 0
for line in attributeList:
    #value_when_true if condition else value_when_false
    numbAttributeReal += int((len(line)-1)/2) if (len(line)-1)/2 != 0.5 else 0
# calculates the number of attributes in the table/matrix -- simple maths

In [71]:
print("Number of attributes detected is: " + str(numbAttributeReal))
# simple print

Number of attributes detected is: 55


In [72]:
attributesNumberList = []
for j in attributeList:
    if len(j) == 1:
        print(j)

In [73]:
# goes through each row(sentence) and extract position of each attribute
attributesNumberList = []
for j in attributeList:
    runningAttributeNum = []
    if len(j) == 2:
        continue # would be like ----- "[sentence] - none" --- no attribute in it
    if j[2] == "All": # if the value is at all then there is only 1 attribute and its the entire text 
        # searchs for the entire text for the entire word
        startChar = totalTextRunning.find(j[0].strip())
        if startChar == -1:
            #checks for the not working -- check function
            startChar = checkNotWorkingValues(j[0].strip(), totalTextRunning)
        if startChar == -1:
            # lower everything and see if it can find it then
            startChar = totalTextRunning.lower().find(j[0].lower().strip())
        endChar = startChar + len(j[0].strip())
        runningAttributeNum.extend([j[1].strip().upper() , startChar, endChar])
        attributesNumberList.append(runningAttributeNum)
    else:
        #print("RUNNING " + str(int((len(j)-1)/2)))
        # im not going to go in detail explaining this but copy this function into a sepreate file and try it out
        # similar to the "all thing" but it works for any number of attributes 
        for w in range(0, int((len(j)-1)), 2):
            runningAttributeNum = []
            if not totalTextRunning.find(j[2+w]) == -1:
                startChar = totalTextRunning.find(j[2+w].strip())
                if startChar == -1:
                    startChar = checkNotWorkingValues(j[2+w].strip(), totalTextRunning)
                if startChar == -1:
                    startChar = totalTextRunning.lower().find(j[2+w].lower().strip())
                endChar = startChar + len(j[2+w])
                runningAttributeNum.extend([j[1+w].upper(), startChar, endChar])
            else:
                words = sentence_2_word(j[2+w]) # for the double check
                detected = 0
                for g in words:
                    if not j[2+w].find(g) == -1:
                        #charTemp = j[2+w].find(g)
                        detected+=1
                if detected/len(words) > 0.95: # split words and 0.95 is threshold
                    startChar = totalTextRunning.find(words[0])
                    endChar = startChar + len(words[0])
                    runningAttributeNum.extend([words[0].upper(), startChar, endChar])
                else:
                    charTemp = -1 # couldn't finf anything
                    print("NOT DETECTED")
                    endChar , startChar = -1
                    runningAttributeNum.extend(["NOTDETECTED", startChar, endChar])
            attributesNumberList.append(runningAttributeNum)

In [74]:
len(attributesNumberList)

55

In [75]:
attributesNumberList

[['NAME', 0, 14],
 ['ADDRESS', 15, 31],
 ['WORKEXPERIENCE', 32, 45],
 ['LINK', 54, 72],
 ['LINK', 83, 112],
 ['PHONE', 113, 127],
 ['LINK', -1, 20],
 ['SOFTWARE', 183, 187],
 ['AWARDCERTIFICATION', 190, 220],
 ['ORGANIZATION', -1, 11],
 ['SOFTWARE', 246, 249],
 ['SOFTWARE', 250, 260],
 ['SOFTWARE', 261, 265],
 ['SOFTWARE', 266, 272],
 ['DATE', 273, 286],
 ['DATE', 289, 299],
 ['WORKEXPERIENCE', 300, 320],
 ['SOFTWARE', 321, 326],
 ['SOFTWARE', 327, 336],
 ['SOFTWARE', 400, 405],
 ['SOFTWARE', 407, 411],
 ['SOFTWARE', 250, 260],
 ['SOFTWARE', 424, 430],
 ['SOFTWARE', 432, 435],
 ['SOFTWARE', 436, 449],
 ['SOFTWARE', 451, 454],
 ['SOFTWARE', 456, 476],
 ['SKILL', 517, 522],
 ['COMPANY', 598, 604],
 ['SKILL', 624, 633],
 ['SKILL', 664, 681],
 ['COMPANY', 682, 699],
 ['ADDRESS', 701, 718],
 ['WORKEXPERIENCE', 719, 726],
 ['DATE', -1, 3],
 ['SKILL', 750, 760],
 ['SKILL', 761, 775],
 ['SKILL', 936, 946],
 ['SKILL', 948, 960],
 ['SKILL', 966, 988],
 ['SKILL', 1009, 1020],
 ['SKILL', 1168, 117

In [76]:
len(attributesNumberList)

55

In [77]:
for testing in attributesNumberList:
    # if it can't find it removes it 
    if testing[1] == -1:
        print(testing)
        attributesNumberList.remove(testing)

['LINK', -1, 20]
['ORGANIZATION', -1, 11]
['DATE', -1, 3]
['ALL', -1, 2]


## DataSet to JSON FILE

In [78]:
len(attributesNumberList)

51

In [79]:
attributeCount = len(attributesNumberList)
text = totalTextRunning
attributes = attributesNumberList

In [80]:
text

'Benjamin Meena\nDenver, CO 80202\nWeb Developer\nGithub: bmeena57.github.io\nLinkedIn: linkedin.com/in/benjaminmeena\n(757) 234-5481\nben.meena16@gmail.co\nm\nEXPERIENCE\nTechnical Highlights\nHTML\n- Certificate of Web\nThe Tech Academy, Denver\nDevelopment\nCSS\nJavaScript\nAJAX\njquery\nNovember 2019 - March 2020\nSoftware Development\nREACT\nBootstrap\nComprehensive overview of software development with courses in HTML5,\nCSS3, JavaScript/jQuery, SQL/MS SQL Server, MVC, Source Control (Git),\nand Agile product development methods (Scrum).\nNon-Technical Highlights\nParticipated in a 2 week long live project with google meetings; updating\nweb pages for their client.\nSelf-driven\nStrong Leadership\n5 Guys Enterprise, Greenwood Village-Manager\nNovember 2019- PRESENT\nManagement\nTime efficient\nEnsure that the Occupational Safety and Health Act, local health and safety codes,\nand company safety and security policies are followed.\nCoachable\nAssist with recruiting, interviewing, 

In [81]:
def createJSONFILE(attributeCount, text, attributes):
    #the 2 string named "totalString" and "attributeString" are just formated in a way how google likes their input value
    # loops through everything and uses the replace function to replace it with all the attributes
    # you have to test this out yourself to understand how it works
    alphab = list(string.ascii_lowercase)
    for alpa in range(int(attributeCount/26)):
        alphab.extend(list(string.ascii_lowercase))
    totalString = """{"textSnippet":{"content":"textCODE"},"annotations":[attributeCODE]}"""
    attributeString = """{"displayName":"AtribnameCODE","textExtraction":{"textSegment":{"startOffset":"sCharCODE","endOffset":"eCharCODE"}}}"""
    totalString =  totalString.replace("textCODE", text)
    tempStringReplacer = ""
    for i in range(attributeCount):
        if i != attributeCount-1:
            tempStringReplacer+=("atriCODE"+str(i+1)+alphab[i]+",")
        else:
            tempStringReplacer+=("atriCODE"+str(i+1)+alphab[i])
    totalString = totalString.replace("attributeCODE", tempStringReplacer)
    for i in range(attributeCount):
        attributeStringNEW = attributeString
        #print(attributes[i])
        attributeStringNEW = attributeStringNEW.replace("AtribnameCODE", attributes[i][0])
        attributeStringNEW = attributeStringNEW.replace("sCharCODE", str(attributes[i][1])).replace("eCharCODE", str(attributes[i][2]))
        totalString = totalString.replace(("atriCODE"+str(i+1)+alphab[i]), attributeStringNEW)
    return totalString

## Final Run Create JSON

In [82]:
file = createJSONFILE(attributeCount, text, attributes)

In [83]:
attributeCount

51

In [84]:
file

'{"textSnippet":{"content":"Benjamin Meena\nDenver, CO 80202\nWeb Developer\nGithub: bmeena57.github.io\nLinkedIn: linkedin.com/in/benjaminmeena\n(757) 234-5481\nben.meena16@gmail.co\nm\nEXPERIENCE\nTechnical Highlights\nHTML\n- Certificate of Web\nThe Tech Academy, Denver\nDevelopment\nCSS\nJavaScript\nAJAX\njquery\nNovember 2019 - March 2020\nSoftware Development\nREACT\nBootstrap\nComprehensive overview of software development with courses in HTML5,\nCSS3, JavaScript/jQuery, SQL/MS SQL Server, MVC, Source Control (Git),\nand Agile product development methods (Scrum).\nNon-Technical Highlights\nParticipated in a 2 week long live project with google meetings; updating\nweb pages for their client.\nSelf-driven\nStrong Leadership\n5 Guys Enterprise, Greenwood Village-Manager\nNovember 2019- PRESENT\nManagement\nTime efficient\nEnsure that the Occupational Safety and Health Act, local health and safety codes,\nand company safety and security policies are followed.\nCoachable\nAssist with

## Save JSONL FILE

In [85]:
os.chdir(jsonFolderPath)
# open a file and save it 
jsonFileName = "FINAL_"+documentRun[9:-4]+".jsonl"
text_file = open(jsonFileName, "w")
text_file.write(file)
text_file.close()

In [86]:
print(jsonFileName)

FINAL_125.jsonl


In [87]:
f = open(jsonFileName, "r")
#print(f.read())

Now your done and you have the Json file, give it to kunal and he will put it into GCP platform account